# Benchmark: NetworkX vs Pandana

This notebook compares the performance of shortest path computations:
- **NetworkX**: General-purpose graph library
- **Pandana**: Specialized for spatial network analysis with precomputation

**Objective**: Demonstrate that Pandana's precomputation approach is significantly faster for repeated spatial queries.

**Author**: VIP Team 1270 - Smart Cities / Urban Systems  
**Project**: Atlanta SDG Portfolio Project


In [ ]:
# Imports
import os
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import pandana as pdna
import networkx as nx

# Set random seed for reproducibility
np.random.seed(42)

print(f"OSMnx version: {ox.__version__}")
print(f"Pandana version: {pdna.__version__}")
print(f"NetworkX version: {nx.__version__}")


In [ ]:
# Set up paths
BASE_DIR = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if 'notebooks' in os.getcwd():
    os.chdir(BASE_DIR)
    
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")


## 1. Download Downtown Atlanta Walk Network


In [ ]:
# Define bounding box around downtown Atlanta
# Using a small area for benchmark (approx 1km x 1km)
DOWNTOWN_LAT = 33.7537
DOWNTOWN_LON = -84.3901
BUFFER_DEG = 0.008  # Approximately 800m

north = DOWNTOWN_LAT + BUFFER_DEG
south = DOWNTOWN_LAT - BUFFER_DEG
east = DOWNTOWN_LON + BUFFER_DEG
west = DOWNTOWN_LON - BUFFER_DEG

print(f"Benchmark area bounding box:")
print(f"  North: {north:.4f}, South: {south:.4f}")
print(f"  East: {east:.4f}, West: {west:.4f}")


In [ ]:
# Download walk network
print("Downloading walk network from OpenStreetMap...")
G = ox.graph_from_bbox(north, south, east, west, network_type='walk')

print(f"\nNetwork downloaded:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")


In [ ]:
# Convert to undirected for walking
G_undirected = ox.convert.to_undirected(G)

# Get GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_undirected)
nodes_gdf = nodes_gdf.reset_index()
edges_gdf = edges_gdf.reset_index()

# Add coordinates
nodes_gdf['x'] = nodes_gdf.geometry.x
nodes_gdf['y'] = nodes_gdf.geometry.y

print(f"Prepared network:")
print(f"  Nodes: {len(nodes_gdf)}")
print(f"  Edges: {len(edges_gdf)}")


## 2. Set Up Benchmark Parameters


In [ ]:
# Select 1 origin and 100 destination nodes (seeded)
all_node_ids = nodes_gdf['osmid'].tolist()

# Pick origin (use first node for stability)
origin_id = all_node_ids[0]

# Pick 100 random destinations
num_destinations = min(100, len(all_node_ids) - 1)
destination_ids = np.random.choice(
    [n for n in all_node_ids if n != origin_id], 
    size=num_destinations, 
    replace=False
)

print(f"Benchmark configuration:")
print(f"  Origin node: {origin_id}")
print(f"  Destination nodes: {num_destinations}")
print(f"  Total path queries: {num_destinations}")


## 3. Create Pandana Network with Precomputation


In [ ]:
# Create Pandana network
print("Creating Pandana network...")

net = pdna.Network(
    nodes_gdf['x'],
    nodes_gdf['y'],
    edges_gdf['u'],
    edges_gdf['v'],
    edges_gdf[['length']],
    twoway=True
)

# Precompute for up to 2km
print("Precomputing shortest paths (up to 2000m)...")
precompute_start = time.time()
net.precompute(2000)
precompute_time = time.time() - precompute_start

print(f"Pandana precomputation complete in {precompute_time:.3f} seconds")


## 4. Benchmark: NetworkX Shortest Paths


In [ ]:
# NetworkX benchmark function
def benchmark_networkx(G, origin, destinations, n_runs=5):
    """Benchmark NetworkX shortest path computation."""
    times = []
    
    for run in range(n_runs):
        start = time.time()
        
        for dest in destinations:
            try:
                length = nx.shortest_path_length(G, origin, dest, weight='length')
            except nx.NetworkXNoPath:
                length = float('inf')
        
        elapsed = time.time() - start
        times.append(elapsed)
    
    return {
        'mean': np.mean(times),
        'std': np.std(times),
        'times': times
    }

print("Running NetworkX benchmark...")
nx_results = benchmark_networkx(G_undirected, origin_id, destination_ids, n_runs=5)
print(f"NetworkX: {nx_results['mean']:.4f}s ± {nx_results['std']:.4f}s")


## 5. Benchmark: Pandana Shortest Paths


In [ ]:
# Pandana benchmark function
def benchmark_pandana(net, nodes_gdf, origin_id, destination_ids, n_runs=5):
    """Benchmark Pandana shortest path computation."""
    times = []
    
    # Get origin coordinates
    origin_row = nodes_gdf[nodes_gdf['osmid'] == origin_id].iloc[0]
    origin_x, origin_y = origin_row['x'], origin_row['y']
    
    # Get destination coordinates
    dest_rows = nodes_gdf[nodes_gdf['osmid'].isin(destination_ids)]
    dest_x = dest_rows['x'].values
    dest_y = dest_rows['y'].values
    
    for run in range(n_runs):
        start = time.time()
        
        # Set origin as POI
        net.set_pois('benchmark_origin', 3000, 1,
                     x_col=pd.Series([origin_x]),
                     y_col=pd.Series([origin_y]))
        
        # Query distances from all destinations to origin
        distances = net.nearest_pois(3000, 'benchmark_origin', num_pois=1)
        
        elapsed = time.time() - start
        times.append(elapsed)
    
    return {
        'mean': np.mean(times),
        'std': np.std(times),
        'times': times
    }

print("Running Pandana benchmark...")
pdna_results = benchmark_pandana(net, nodes_gdf, origin_id, destination_ids, n_runs=5)
print(f"Pandana: {pdna_results['mean']:.4f}s ± {pdna_results['std']:.4f}s")


## 6. Results Summary


In [ ]:
# Calculate speedup
speedup = nx_results['mean'] / pdna_results['mean']

print("="*60)
print("BENCHMARK RESULTS")
print("="*60)
print(f"\nNetwork size: {len(nodes_gdf)} nodes, {len(edges_gdf)} edges")
print(f"Queries: {num_destinations} destination nodes")
print(f"\n{'Method':<15} {'Mean Time':>12} {'Std Dev':>12} {'Speedup':>10}")
print("-"*50)
print(f"{'NetworkX':<15} {nx_results['mean']:>10.4f}s {nx_results['std']:>10.4f}s {'1.0x':>10}")
print(f"{'Pandana':<15} {pdna_results['mean']:>10.4f}s {pdna_results['std']:>10.4f}s {speedup:>9.1f}x")
print("="*60)
print(f"\nPandana is {speedup:.1f}x faster than NetworkX for this benchmark.")


In [ ]:
# Export results to markdown file
results_md = f"""# Benchmark Results: NetworkX vs Pandana

## Test Configuration

| Parameter | Value |
|-----------|-------|
| Network nodes | {len(nodes_gdf)} |
| Network edges | {len(edges_gdf)} |
| Origin nodes | 1 |
| Destination nodes | {num_destinations} |
| Benchmark runs | 5 |
| Random seed | 42 |

## Results

| Method | Mean Time (s) | Std Dev (s) | Speedup |
|--------|---------------|-------------|---------|
| NetworkX | {nx_results['mean']:.4f} | {nx_results['std']:.4f} | 1.0x |
| Pandana | {pdna_results['mean']:.4f} | {pdna_results['std']:.4f} | {speedup:.1f}x |

## Key Finding

**Pandana is {speedup:.1f}x faster** than NetworkX for repeated shortest path queries.

### Why is Pandana faster?

1. **Precomputation**: Pandana precomputes shortest paths up to a specified distance, making subsequent queries O(1).
2. **Specialized data structures**: Designed specifically for spatial network analysis.
3. **C++ backend**: Core algorithms implemented in optimized C++.

### When to use each:

- **NetworkX**: General graph analysis, small networks, one-off computations
- **Pandana**: Spatial accessibility analysis, repeated queries, large networks

---

*Generated by Atlanta SDG Portfolio Project - VIP Team 1270*
"""

# Save to file
results_path = os.path.join(OUTPUT_DIR, 'benchmark_results.md')
with open(results_path, 'w') as f:
    f.write(results_md)

print(f"Results saved to: {results_path}")


In [ ]:
# Display the markdown results
from IPython.display import Markdown, display
display(Markdown(results_md))
